<a href="https://colab.research.google.com/github/mathluva/TwitterSentiment-BERT/blob/main/SentimentTwitter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Sun Feb 28 09:43:49 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import pandas as pd
import numpy as np
import torch
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split

In [3]:
from google.colab import files
uploaded = files.upload()

Saving smile-annotations-final.csv to smile-annotations-final.csv


In [55]:
import io
df = pd.read_csv(io.BytesIO(uploaded['smile-annotations-final.csv']))
df.head()

,611857364396965889,@aandraous @britishmuseum @AndrewsAntonio Merci pour le partage! @openwinemap,nocode
0,614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy
1,614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy
2,614877582664835073,@Sofabsports thank you for following me back. ...,happy
3,611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy
4,611570404268883969,@NationalGallery @ThePoldarkian I have always ...,happy


In [56]:
df = df.rename(columns={'611857364396965889': 'id','@aandraous @britishmuseum @AndrewsAntonio Merci pour le partage! @openwinemap': 'text', 'nocode':'label'})
df.head()

,id,text,label
0,614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy
1,614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy
2,614877582664835073,@Sofabsports thank you for following me back. ...,happy
3,611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy
4,611570404268883969,@NationalGallery @ThePoldarkian I have always ...,happy


In [57]:
rows = len(df)
print(f'There are {rows} rows')

There are 3084 rows


In [58]:
df.tail(20)

,id,text,label
3064,610575620641300480,Big up to #kayaktivists @cmkshama #Seattle #S...,nocode
3065,614089268198285312,Fantastic news for #digitalpreservation as @dr...,happy
3066,611537640857411584,@britishmuseum @SenderosP The Rosetta Stone ;),happy
3067,611227963976253440,THIS is why #ilovemuseums ! Torres Strait danc...,happy
3068,614074786042486785,I wonder what hitherto unknown treasures might...,nocode
3069,612242969035411456,@britishmuseum on INSTA. See this magnificent ...,happy
3070,614900716960915456,This was definitely one of the best cocktails ...,happy
3071,611557802197299201,James Gillray (1756-1815) @britishmuseum until...,nocode
3072,611532293358899200,@DerekSharplol @britishmuseum How about Beef W...,nocode
3073,613646155591909377,#Caravaggio painted the beheading of St John t...,nocode


In [59]:
#replace index with id
df.set_index('id', inplace =True) 

In [60]:
#check sum of identity_hate
df.label.value_counts()

nocode               1571
happy                1137
not-relevant          214
angry                  57
surprise               35
sad                    32
happy|surprise         11
happy|sad               9
disgust|angry           7
disgust                 6
sad|angry               2
sad|disgust             2
sad|disgust|angry       1
Name: label, dtype: int64

ValueError: ignored

In [ ]:
#create new column to hold label value


possible_lables = ['identity_hate', 'insult', 'obscene', 'severe_toxic', 'threat', 'toxic']

label_dict = {}
for index, label in enumerate(possible_labels):
    label_dict[label] = index

df['label_value'] = df.label.replace(label_dict)

In [ ]:
df.tail(10)

,comment_text,identity_hate,insult,obscene,set,severe_toxic,threat,toxic,toxicity,label,label_value
id,,,,,,,,,,,
fb4cbf4eeabe23d4,study some linguistics before you say somethi...,0.0,0.0,0.0,train,0.0,0.0,1.0,1.0,toxic,5
fbb9d76ad8911cc4,the first of your links is something to agree ...,0.0,0.0,1.0,train,0.0,0.0,0.0,1.0,obscene,2
fbf8672ea3b4ddf7,http w nysun com article public interest crap,0.0,0.0,0.0,train,0.0,0.0,1.0,1.0,toxic,5
fc6d45d108129fc8,goethean and me i would like you to know i ha...,0.0,0.0,0.0,train,0.0,0.0,1.0,1.0,toxic,5
fccf0939631ab7c8,stop telling lies and trying to promote your p...,0.0,0.0,0.0,train,0.0,0.0,1.0,1.0,toxic,5
fcf5a6ad5918f164,your boring and retard ed two,0.0,0.0,0.0,train,0.0,0.0,1.0,1.0,toxic,5
fd0129fde97321cb,why did that idiot revert the reversion i made...,0.0,0.0,0.0,train,0.0,0.0,1.0,1.0,toxic,5
fd68ef478b3dfd05,ps you re all middle aged losers at home in yo...,0.0,0.0,0.0,train,0.0,0.0,1.0,1.0,toxic,5
fdc92e571d39e7e1,yeah i no it suck s,0.0,0.0,0.0,train,0.0,0.0,1.0,1.0,toxic,5


In [ ]:
#check totals
df['label'].value_counts()

toxic            5666
obscene           317
insult            301
identity_hate      54
threat             22
Name: label, dtype: int64

In [ ]:
df = df.drop(columns= ['set', 'toxicity'])
df.head()

,comment_text,identity_hate,insult,obscene,severe_toxic,threat,toxic,label,label_value
id,,,,,,,,,
0005c987bdfc9d4b,hey what is it talk what is it an exclusive ...,0.0,0.0,0.0,0.0,0.0,1.0,toxic,5
0007e25b2121310b,bye don t look come or think of comming back...,0.0,0.0,0.0,0.0,0.0,1.0,toxic,5
001956c382006abd,i m sorry i m sorry i screwed around with some...,0.0,0.0,0.0,0.0,0.0,1.0,toxic,5
002264ea4d5f2887,why can t you believe how fat artie is did yo...,0.0,0.0,0.0,0.0,0.0,1.0,toxic,5
005f59485fcddeb0,sorry puck but no one ever said dick was numb...,0.0,0.0,0.0,0.0,0.0,1.0,toxic,5


In [ ]:
#use stratified sampling to ensure that sampled dataset is representative of the entire population
X_train, X_val, y_train, y_val= train_test_split(df.index.values, df.label_value.values, random_state = 42, stratify = df['label'].values)


In [ ]:
#create set column
#locate training and val samples
df['sample_type'] = ['none']*df.shape[0]

df.loc[X_train, 'sample_type'] = 'train'
df.loc[X_val, 'sample_type'] = 'val'

In [ ]:
#check stratified sampling values

df.groupby(['label', 'label_value', 'sample_type']).count()

comment_text  ...  toxic
label         label_value sample_type                ...       
identity_hate 0           train                  40  ...     40
                          val                    14  ...     14
insult        1           train                 226  ...    226
                          val                    75  ...     75
obscene       2           train                 238  ...    238
                          val                    79  ...     79
threat        4           train                  17  ...     17
                          val                     5  ...      5
toxic         5           train                4249  ...   4249
                          val                  1417  ...   1417

[10 rows x 7 columns]

In [ ]:
!pip install transformers==3.5

In [ ]:
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case = True)


In [ ]:
#encode train data
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.sample_type=='train'].comment_text.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    max_length = 256,
    padding = True,
    truncation = True,
    return_tensors='pt'
)

input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.sample_type=='train'].label_value.values)



In [ ]:
#encode validation data
encoded_data_val = tokenizer.batch_encode_plus(
    df[df.sample_type=='val'].comment_text.values, 
   add_special_tokens=True, 
    return_attention_mask=True, 
    max_length = 256,
    padding = True,
    truncation = True,
    return_tensors='pt'
)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.sample_type=='val'].label_value.values)

In [ ]:
# Convert all inputs and labels into torch tensors
#BERT model required datatype 
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [ ]:
print(len(dataset_train))
print(len(dataset_val))

4770
1590


In [ ]:
from transformers import BertForSequenceClassification

In [ ]:
 model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                       num_labels=len(possible_labels),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

In [ ]:
batch_size = 32

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup

In [ ]:
optimizer = AdamW(model.parameters())

In [ ]:
epochs = 10

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

In [ ]:
def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [ ]:
import random
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cuda


In [ ]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [ ]:
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')


Epoch 1
Training loss: 0.5326458472510179
Validation loss: 0.4970031551271677
F1 Score (Weighted): 0.8399223614501173



Epoch 2
Training loss: 0.4851704913377762
Validation loss: 0.46577891454100606
F1 Score (Weighted): 0.8399223614501173



Epoch 3
Training loss: 0.47710482935110726
Validation loss: 0.48643613487482074
F1 Score (Weighted): 0.8399223614501173



Epoch 4
Training loss: 0.48505168572068214
Validation loss: 0.46631444901227953
F1 Score (Weighted): 0.8399223614501173



Epoch 5
Training loss: 0.4734137836595376
Validation loss: 0.47372164085507396
F1 Score (Weighted): 0.8399223614501173



Epoch 6
Training loss: 0.47302221131821476
Validation loss: 0.46492910400033
F1 Score (Weighted): 0.8399223614501173



Epoch 7
Training loss: 0.476940329571565
Validation loss: 0.4598277512192726
F1 Score (Weighted): 0.8399223614501173



Epoch 8
Training loss: 0.4653632979094982
Validation loss: 0.4685015004873276
F1 Score (Weighted): 0.8399223614501173



Epoch 9
Training loss: 0.4637651174267133
Validation loss: 0.46462376579642295
F1 Score (Weighted): 0.8399223614501173



Epoch 10
Training loss: 0.463958105246226
Validation loss: 0.45730825573205947
F1 Score (Weighted): 0.8399223614501173



In [ ]:
_, predictions, true_vals = evaluate(dataloader_validation)

In [ ]:
accuracy_per_class(predictions, true_vals)

Class: identity_hate
Accuracy: 4/14

Class: insult
Accuracy: 2/75

Class: obscene
Accuracy: 2/79

Class: threat
Accuracy: 4/5

Class: toxic
Accuracy: 371/1417



In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
model_save_name = 'classifierII.pt'
path = F"/content/gdrive/My Drive/{model_save_name}" 
torch.save(model.state_dict(), path)